# Hybridize: 更快和更好 
到目前为止我们看到的教程都使用了命令式的编程。你可能之前都从来没有听说过这个词。不过，一直以来我们都是用这个方式写python代码。 

考虑下面这段代码：


In [1]:
def add(A,B):
    return A+B 

def add_str():
    return '''
def add(A,B):
    return A+B
'''

def fancy_func_str():
    return '''
def fancy_func(A,B,C,D):
    E=add(A,B)
    F=add(C,D)
    G=add(E,F)
    return G 
'''
def evoke_str():
    return add_str()+fancy_func_str()+'''
print(fancy_func(1,2,3,4)) 
'''
 
prog=evoke_str()
y=compile(prog,'','exec')
exec(y) 

10


In [2]:
from mxnet.gluon import nn 
from mxnet import nd 
def get_net():
    net=nn.HybridSequential()
    with net.name_scope():
        net.add(
            nn.Dense(256,activation='relu'),
            nn.Dense(256,activation='relu'),
            nn.Dense(2)
        )
    net.initialize() 
    return net 
x=nd.random.normal(shape=(1,512)) 
net=get_net()
net(x) 


[[-0.09420323 -0.07313035]]
<NDArray 1x2 @cpu(0)>

我们可以通过hybridize 来编译优化HybridSequential。 

In [3]:
net.hybridize() 
net(x) 


[[-0.09420323 -0.07313035]]
<NDArray 1x2 @cpu(0)>

注意到只有继承自HybridBlock的层才会被优化。HybridSequential和Gluon提供的子类。如果一个层知识继承自Block,那么我们将跳过优化

# 性能
我们比较hybridize 前后的计算时间来展示符号式执行的性能提升。
我们这里计算1000次 forward:

In [4]:
from time import time 
def bench(net,x):
    start=time() 
    for i in range(1000):
        y=net(x)
    # 等待计算完成 
    nd.waitall() 
    return time()-start 

net=get_net() 
print("before hybridizing :%.4f sec" %(bench(net,x)))
net.hybridize() 
print("After hybridizing: %.4f sec "%(bench(net,x))) 

before hybridizing :0.4919 sec
After hybridizing: 0.3576 sec 


可以看到hybridize提供近两倍的加速 
# 获取符号式的程序 
之前我们给net 输入NDArray类型的x,然后net(x) 会直接返回结果。对于调用过的hybridize()后的网络，我们可以给它输入一个Symbol类型的变量，其会返回同样是Symbol类型的程序 。 

In [5]:
from mxnet import sym 
x=sym.var('data') 
y=net(x) 
print(y.tojson())

{
  "nodes": [
    {
      "op": "null", 
      "name": "data", 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential1_dense0_weight", 
      "attrs": {
        "__dtype__": "0", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256, 0)", 
        "__storage_type__": "0", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "null", 
      "name": "hybridsequential1_dense0_bias", 
      "attrs": {
        "__dtype__": "0", 
        "__init__": "zeros", 
        "__lr_mult__": "1.0", 
        "__shape__": "(256,)", 
        "__storage_type__": "0", 
        "__wd_mult__": "1.0"
      }, 
      "inputs": []
    }, 
    {
      "op": "FullyConnected", 
      "name": "hybridsequential1_dense0_fwd", 
      "attrs": {
        "flatten": "True", 
        "no_bias": "False", 
        "num_hidden": "256"
      }, 
      "inputs": [[0, 0, 0], [1, 0, 0], [2, 0, 0]]
    }, 
    {
      "op": "Activation", 
      "name": "hybridse



我们可以通过`export()`来保存这个程序到硬盘。它可以之后不仅被Python，同时也可以其他支持的前端语言，例如C++, Scala, R...，读取。

TODO(mli) `export`需要`mxnet>=0.11.1b20171015`，样例之后放进来。

## 通过HybridBlock深入理解`hybridize`工作机制

前面我们展示了通过`hybridize`我们可以获得更好的性能和更高的移植性。现在我们来解释这个是如何影响灵活性的。记得我们提过gluon里面的`Sequential`是`Block`的一个便利形式，同理，可以`HybridSequential`是`HybridBlock`的子类。跟`Block`需要实现`forward`方法不一样，对于`HybridBlock`我们需要实现`hybrid_forward`方法。


In [6]:
class HybridNet(nn.HybridBlock):
    def __init__(self,**kwargs):
        super(HybridNet,self).__init__(**kwargs)
        with self.name_scope():
            self.fc1=nn.Dense(10) 
            self.fc2=nn.Dense(2) 
    def hybrid_forward(self,F,x):
        print(F) 
        print(x) 
        x=F.relu(self.fc1(x)) 
        print(x) 
        return self.fc2(x) 

`hybrid_forward`方法加入了额外的输入`F`，它使用了MXNet的一个独特的特征。MXNet有一个符号式的API (`symbol`) 和命令式的API (`ndarray`)。这两个接口里面的函数基本是一致的。系统会根据输入来决定`F`是使用`symbol`还是`ndarray`。

我们实例化一个样例，然后可以看到默认`F`是使用`ndarray`。而且我们打印出了输入和第一层relu的输出。

In [8]:
net=HybridNet()
net.initialize() 
x=nd.random.normal(shape=(1,4)) 
y=net(x) 
y

<module 'mxnet.ndarray' from '/home/dske/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/ndarray/__init__.py'>

[[ 1.3597034   0.77690774 -0.6409365   0.25685814]]
<NDArray 1x4 @cpu(0)>

[[0.03664961 0.         0.01281041 0.08815128 0.         0.0844786
  0.00965692 0.1277262  0.05697468 0.00549724]]
<NDArray 1x10 @cpu(0)>



[[ 0.00544146 -0.01244316]]
<NDArray 1x2 @cpu(0)>

接下来看看hybridze 后会发生什么。

In [9]:
net.hybridize() 
y=net(x) 

<module 'mxnet.symbol' from '/home/dske/anaconda3/envs/gluon/lib/python3.6/site-packages/mxnet/symbol/__init__.py'>
<Symbol data>
<Symbol hybridnet1_relu0>




可以看到：

1. `F`变成了`symbol`.
2. 即使输入数据还是`NDArray`的类型，但`hybrid_forward`里不论是输入还是中间输出，全部变成了`Symbol`

再运行一次看看